In [1]:
import csv

import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

RANDOM_SEED = 42

2025-06-01 21:21:36.944894: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-01 21:21:37.391443: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-01 21:21:37.876552: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748802098.167666    5425 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748802098.280752    5425 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748802099.158879    5425 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

# Specify each path

In [2]:
dataset = 'model/keypoint_classifier/keypoint.csv'
model_save_path = 'model/keypoint_classifier/keypoint_classifier.keras'
tflite_save_path = 'model/keypoint_classifier/keypoint_classifier.tflite'

# Set number of classes

In [3]:
NUM_CLASSES = 13

# Dataset reading

In [4]:
X_dataset = np.loadtxt(dataset, delimiter=',', dtype='float32', usecols=list(range(1, (21 * 2) + 1)))

In [5]:
y_dataset = np.loadtxt(dataset, delimiter=',', dtype='int32', usecols=(0))

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X_dataset, y_dataset, train_size=0.75, random_state=RANDOM_SEED)

# Model building

In [7]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input((21 * 2, )),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(20, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(10, activation='relu'),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])

2025-06-01 21:21:47.740603: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [8]:
model.summary()  # tf.keras.utils.plot_model(model, show_shapes=True)

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dropout (Dropout)               │ (None, 42)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 20)             │           860 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 20)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │           210 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 13)             │           143 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,213 (4.74 KB)

 Trainable params: 1,213 (4.74 KB)

 Non-trainable params: 0 (0.00 B)

In [9]:
# Model checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    model_save_path, verbose=1, save_weights_only=False)
# Callback for early stopping
es_callback = tf.keras.callbacks.EarlyStopping(patience=20, verbose=1)

In [10]:
# Model compilation
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Model training

In [11]:
model.fit(
    X_train,
    y_train,
    epochs=1000,
    batch_size=128,
    validation_data=(X_test, y_test),
    callbacks=[cp_callback, es_callback]
)

Epoch 1/1000
46/48 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.1850 - loss: 2.4904
Epoch 1: saving model to model/keypoint_classifier/keypoint_classifier.keras
48/48 ━━━━━━━━━━━━━━━━━━━━ 2s 14ms/step - accuracy: 0.1862 - loss: 2.4858 - val_accuracy: 0.2484 - val_loss: 2.1989
Epoch 2/1000
38/48 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2345 - loss: 2.2453
Epoch 2: saving model to model/keypoint_classifier/keypoint_classifier.keras
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.2370 - loss: 2.2339 - val_accuracy: 0.3081 - val_loss: 1.9996
Epoch 3/1000
37/48 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2511 - loss: 2.0887
Epoch 3: saving model to model/keypoint_classifier/keypoint_classifier.keras
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.2558 - loss: 2.0768 - val_accuracy: 0.3255 - val_loss: 1.8304
Epoch 4/1000
41/48 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.2915 - loss: 1.9308
Epoch 4: saving model to model/keypoint_classifier/keypoint_classifier.keras
48

Epoch 25/1000
24/48 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5440 - loss: 1.1422 
Epoch 25: saving model to model/keypoint_classifier/keypoint_classifier.keras
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5399 - loss: 1.1509 - val_accuracy: 0.7944 - val_loss: 0.8090
Epoch 26/1000
47/48 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5484 - loss: 1.1550
Epoch 26: saving model to model/keypoint_classifier/keypoint_classifier.keras
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5483 - loss: 1.1550 - val_accuracy: 0.7855 - val_loss: 0.8026
Epoch 27/1000
28/48 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5645 - loss: 1.1313 
Epoch 27: saving model to model/keypoint_classifier/keypoint_classifier.keras
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5616 - loss: 1.1323 - val_accuracy: 0.7929 - val_loss: 0.7853
Epoch 28/1000
43/48 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5402 - loss: 1.1802
Epoch 28: saving model to model/keypoint_classifier/keypoint_classifier

48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - accuracy: 0.6188 - loss: 1.0187 - val_accuracy: 0.8631 - val_loss: 0.6277
Epoch 49/1000
47/48 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.6190 - loss: 0.9956
Epoch 49: saving model to model/keypoint_classifier/keypoint_classifier.keras
48/48 ━━━━━━━━━━━━━━━━━━━━ 3s 65ms/step - accuracy: 0.6188 - loss: 0.9959 - val_accuracy: 0.8571 - val_loss: 0.6117
Epoch 50/1000
47/48 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.6139 - loss: 1.0037
Epoch 50: saving model to model/keypoint_classifier/keypoint_classifier.keras
48/48 ━━━━━━━━━━━━━━━━━━━━ 3s 58ms/step - accuracy: 0.6142 - loss: 1.0036 - val_accuracy: 0.8686 - val_loss: 0.6082
Epoch 51/1000
46/48 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.6228 - loss: 0.9859
Epoch 51: saving model to model/keypoint_classifier/keypoint_classifier.keras
48/48 ━━━━━━━━━━━━━━━━━━━━ 2s 34ms/step - accuracy: 0.6229 - loss: 0.9864 - val_accuracy: 0.8756 - val_loss: 0.6042
Epoch 52/1000
48/48 ━━━━━━━━━━━━━━━━━━

Epoch 72/1000
44/48 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6268 - loss: 0.9766
Epoch 72: saving model to model/keypoint_classifier/keypoint_classifier.keras
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.6280 - loss: 0.9742 - val_accuracy: 0.8865 - val_loss: 0.5533
Epoch 73/1000
44/48 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6378 - loss: 0.9781
Epoch 73: saving model to model/keypoint_classifier/keypoint_classifier.keras
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.6377 - loss: 0.9774 - val_accuracy: 0.8850 - val_loss: 0.5495
Epoch 74/1000
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.6398 - loss: 0.9566
Epoch 74: saving model to model/keypoint_classifier/keypoint_classifier.keras
48/48 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.6398 - loss: 0.9565 - val_accuracy: 0.8850 - val_loss: 0.5563
Epoch 75/1000
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6417 - loss: 0.9499
Epoch 75: saving model to model/keypoint_classifier/keypoint_classi

48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6663 - loss: 0.9041 - val_accuracy: 0.8800 - val_loss: 0.5266
Epoch 96/1000
26/48 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6419 - loss: 0.9650 
Epoch 96: saving model to model/keypoint_classifier/keypoint_classifier.keras
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6478 - loss: 0.9512 - val_accuracy: 0.8880 - val_loss: 0.5121
Epoch 97/1000
47/48 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6510 - loss: 0.9371
Epoch 97: saving model to model/keypoint_classifier/keypoint_classifier.keras
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6512 - loss: 0.9371 - val_accuracy: 0.8840 - val_loss: 0.5064
Epoch 98/1000
22/48 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6696 - loss: 0.8868 
Epoch 98: saving model to model/keypoint_classifier/keypoint_classifier.keras
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6634 - loss: 0.9012 - val_accuracy: 0.8920 - val_loss: 0.4988
Epoch 99/1000
40/48 ━━━━━━━━━━━━━━━━━━━━ 0s

Epoch 119/1000
33/48 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6829 - loss: 0.8743 
Epoch 119: saving model to model/keypoint_classifier/keypoint_classifier.keras
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6811 - loss: 0.8774 - val_accuracy: 0.8815 - val_loss: 0.4889
Epoch 120/1000
29/48 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6606 - loss: 0.9013 
Epoch 120: saving model to model/keypoint_classifier/keypoint_classifier.keras
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6620 - loss: 0.9031 - val_accuracy: 0.8805 - val_loss: 0.4880
Epoch 121/1000
29/48 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6665 - loss: 0.9154 
Epoch 121: saving model to model/keypoint_classifier/keypoint_classifier.keras
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6651 - loss: 0.9151 - val_accuracy: 0.8771 - val_loss: 0.5073
Epoch 122/1000
30/48 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6652 - loss: 0.8947 
Epoch 122: saving model to model/keypoint_classifier/keypoint_

31/48 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6764 - loss: 0.8826 
Epoch 142: saving model to model/keypoint_classifier/keypoint_classifier.keras
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6762 - loss: 0.8875 - val_accuracy: 0.8795 - val_loss: 0.4912
Epoch 143/1000
33/48 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6851 - loss: 0.8706 
Epoch 143: saving model to model/keypoint_classifier/keypoint_classifier.keras
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6810 - loss: 0.8784 - val_accuracy: 0.8805 - val_loss: 0.4897
Epoch 144/1000
32/48 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6827 - loss: 0.8561 
Epoch 144: saving model to model/keypoint_classifier/keypoint_classifier.keras
48/48 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6826 - loss: 0.8571 - val_accuracy: 0.8726 - val_loss: 0.4919
Epoch 145/1000
42/48 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6578 - loss: 0.8996
Epoch 145: saving model to model/keypoint_classifier/keypoint_classifier.keras

In [12]:
# Model evaluation
val_loss, val_acc = model.evaluate(X_test, y_test, batch_size=128)

16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8804 - loss: 0.4823 


In [13]:
# Loading the saved model
model = tf.keras.models.load_model(model_save_path)

In [14]:
# Inference test
predict_result = model.predict(np.array([X_test[0]]))
print(np.squeeze(predict_result))
print(np.argmax(np.squeeze(predict_result)))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
[4.7132891e-02 9.4836242e-03 2.7475543e-02 1.4167111e-02 3.8806188e-06
 6.8518430e-01 2.3688548e-05 2.1652098e-01 4.6758823e-06 1.6831843e-06
 4.2940505e-07 3.9630123e-07 7.8286462e-07]
5


# Confusion matrix

In [15]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report

def print_confusion_matrix(y_true, y_pred, report=True):
    labels = sorted(list(set(y_true)))
    cmx_data = confusion_matrix(y_true, y_pred, labels=labels)
    
    df_cmx = pd.DataFrame(cmx_data, index=labels, columns=labels)
 
    fig, ax = plt.subplots(figsize=(7, 6))
    sns.heatmap(df_cmx, annot=True, fmt='g' ,square=False)
    ax.set_ylim(len(set(y_true)), 0)
    plt.show()
    
    if report:
        print('Classification Report')
        print(classification_report(y_test, y_pred))

Y_pred = model.predict(X_test)
y_pred = np.argmax(Y_pred, axis=1)

print_confusion_matrix(y_test, y_pred)

ModuleNotFoundError: No module named 'pandas'

# Convert to model for Tensorflow-Lite

In [ ]:
# Save as a model dedicated to inference
model.save(model_save_path, include_optimizer=False)

In [ ]:
# Transform model (quantization)

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quantized_model = converter.convert()

open(tflite_save_path, 'wb').write(tflite_quantized_model)

# Inference test

In [ ]:
interpreter = tf.lite.Interpreter(model_path=tflite_save_path)
interpreter.allocate_tensors()

In [ ]:
# Get I / O tensor
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [ ]:
interpreter.set_tensor(input_details[0]['index'], np.array([X_test[0]]))

In [ ]:
%%time
# Inference implementation
interpreter.invoke()
tflite_results = interpreter.get_tensor(output_details[0]['index'])

In [ ]:
print(np.squeeze(tflite_results))
print(np.argmax(np.squeeze(tflite_results)))